In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]= "true"

import numpy as np
from tensorflow.keras.optimizers import Adam , RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , concatenate , Conv2D , MaxPooling2D , Activation , UpSampling2D , BatchNormalization, Conv2DTranspose
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import tensorflow as tf

from tqdm.notebook import tqdm

import requests
import json

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
# インプット画像を読み込む関数
def load_X_gray(folder_path):
    
    image_files = []

    #image_files = os.listdir(folder_path)
       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    #image_files = image_files[1:]
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        #image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    
    print(images.shape)
    
    return images, image_files


def load_Y_gray(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in tqdm(enumerate(image_files)):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def divide_imgs(images):
    
    '''
        512 x 512に分割する
        端の50pxをのりしろとする
        空欄箇所は-1（黒）にする
        height, widthが412未満の場合は変数を-1に変更
    '''
    
    H = -(-images.shape[1]//412)
    W = -(-images.shape[2]//412)
    
    diveded_imgs = np.zeros(( images.shape[0]*H*W, 512, 512, 1), np.float32)
    print(H,W)
    
    for z in range(images.shape[0]):
        image = images[z]
        for h in range(H):
            for w in range(W):
                cropped_img = np.zeros((512, 512, 1), np.float32)
                cropped_img -= 1
                
                if images.shape[1] < 412:
                    h = -1
                if images.shape[2] < 412:
                    w = -1
                    
                if h == -1:
                    if w == -1:
                        cropped_img[50:images.shape[1]+50, 50:images.shape[2]+50, 0] = image[0:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:images.shape[1]+50, 50:512, 0] = image[0:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:images.shape[1]+50, 0:images.shape[2]-412*W-50, 0] = image[0:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        cropped_img[50:images.shape[1]+50, :, 0] = image[0:images.shape[1], w*412-50:(w+1)*412+50, 0]
                elif h == 0:
                    if w == -1:
                        cropped_img[50:512, 50:images.shape[2]+50, 0] = image[0:462, 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:512, 50:512, 0] = image[0:462, 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:512, 0:images.shape[2]-412*W-50, 0] = image[0:462, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[50:512, 0:images.shape[2]-412*(W-1)-50, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                elif h == H-1:
                    if w == -1:
                        cropped_img[0:images.shape[1]-412*H-50, 50:images.shape[2]+50, 0] = image[h*412-50:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[0:images.shape[1]-412*H-50, 50:512, 0] = image[h*412-50:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[0:images.shape[1]-412*H-50, :, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                else:
                    if w == -1:
                        cropped_img[:, 50:images.shape[2]+50, 0] = image[h*412-50:(h+1)*412+50, 0:images.shape[2], 0]
                    elif w == 0:
                        #cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        try:
                            cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                    elif w == W-1:
                        #cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        try:
                            cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                    else:
                        #cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                        try:
                            cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]         
                        except:
                            try:
                                 cropped_img[:, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                            except:
                                 cropped_img[0:images.shape[1]-412*(H-1)-50, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                h = max(0, h)
                w = max(0, w)
                diveded_imgs[z*H*W+ w*H+h] = cropped_img
                #print(z*H*W+ w*H+h)
                
    return diveded_imgs
    
    
    
    
def merge_imgs(imgs, original_image_shape):
    
    '''
        元のサイズに合体させる
        original_image_shapeはoriginal_image.shapeで与えられる、(z, h, w, channel)のタプル型
    '''
    
    merged_imgs = np.zeros((original_image_shape[0], original_image_shape[1], original_image_shape[2], 1), np.float32)
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412)    
    
    for z in range(original_image_shape[0]):
        for h in range(H):
            for w in range(W):

                if original_image_shape[1] < 412:
                    h = -1
                if original_image_shape[2] < 412:
                    w = -1
                    
                #print(z*H*W+ max(w, 0)*H+max(h, 0))    
                if h == -1:
                    if w == -1:
                        merged_imgs[z, 0:original_image_shape[1], 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+0][50:original_image_shape[1]+50, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, 0:original_image_shape[1], 0:412, 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, 0:original_image_shape[1], w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, 0:original_image_shape[1], w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:462, 0]
                elif h == 0:
                    if w == -1:
                        merged_imgs[z, 0:412, 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:462, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, 0:412, 0:412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, 0:412, w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:462, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, 0:412, w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                elif h == H-1:
                    if w == -1:
                         merged_imgs[z, h*412:original_image_shape[1], 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:original_image_shape[1]-412*H-50, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, h*412:original_image_shape[1], 0:412, 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, h*412:original_image_shape[1], w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, h*412:original_image_shape[1], w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:462, 0]
                else:
                    if w == -1:
                         merged_imgs[z, h*412:(h+1)*412, 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:462, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, h*412:(h+1)*412, 0:412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, h*412:(h+1)*412, w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:462, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, h*412:(h+1)*412, w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]  
        
    print(merged_imgs.shape)
    return merged_imgs

In [ ]:
def standard_uint(input_tensor, nb_filter):
    
    x = Conv2D(nb_filter, (3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x


def get_nest_unet_512_layer_7(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)
        pool5 = MaxPooling2D((2, 2), strides = (2, 2))(conv5_1)      
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        # 16
        conv6_1 = standard_uint(pool5, nb_filter = 512)        
        
        up5_2 = Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv6_1)
        conv5_2 = concatenate([up5_2, conv5_1], axis = 3)
        conv5_2 = standard_uint(conv5_2, nb_filter = 256) 
        
        up4_3 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_2)
        conv4_3 = concatenate([up4_3, conv4_1, conv4_2], axis = 3)
        conv4_3 = standard_uint(conv4_3, nb_filter = 128)
        
        up3_4 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_3)
        conv3_4 = concatenate([up3_4, conv3_1, conv3_2, conv3_3], axis = 3)
        conv3_4 = standard_uint(conv3_4, nb_filter = 64)
        
        up2_5 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_4)
        conv2_5 = concatenate([up2_5, conv2_1, conv2_2, conv2_3, conv2_4], axis = 3)
        conv2_5 = standard_uint(conv2_5, nb_filter = 32)   
        
        up1_6 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_5)
        conv1_6 = concatenate([up1_6, conv1_1, conv1_2, conv1_3, conv1_4, conv1_5], axis = 3)
        conv1_6 = standard_uint(conv1_6, nb_filter = 16)   
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        nested_output_5 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_6)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4, nested_output_5]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_5])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model
    
   

def get_nest_unet_512_layer_6(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)
        pool5 = MaxPooling2D((2, 2), strides = (2, 2))(conv5_1)      
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        # 16
        conv6_1 = standard_uint(pool5, nb_filter = 512)
        pool6 = MaxPooling2D((2, 2), strides = (2, 2))(conv6_1)        
        
        up5_2 = Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv6_1)
        conv5_2 = concatenate([up5_2, conv5_1], axis = 3)
        conv5_2 = standard_uint(conv5_2, nb_filter = 256) 
        
        up4_3 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_2)
        conv4_3 = concatenate([up4_3, conv4_1, conv4_2], axis = 3)
        conv4_3 = standard_uint(conv4_3, nb_filter = 128)
        
        up3_4 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_3)
        conv3_4 = concatenate([up3_4, conv3_1, conv3_2, conv3_3], axis = 3)
        conv3_4 = standard_uint(conv3_4, nb_filter = 64)
        
        up2_5 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_4)
        conv2_5 = concatenate([up2_5, conv2_1, conv2_2, conv2_3, conv2_4], axis = 3)
        conv2_5 = standard_uint(conv2_5, nb_filter = 32)   
        
        up1_6 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_5)
        conv1_6 = concatenate([up1_6, conv1_1, conv1_2, conv1_3, conv1_4, conv1_5], axis = 3)
        conv1_6 = standard_uint(conv1_6, nb_filter = 16)   
        
        # Center
        conv7_1 = standard_uint(pool6, nb_filter = 1024)
        
        up6_2 = Conv2DTranspose(512, (2, 2), strides = (2, 2), padding = 'same')(conv7_1)
        conv6_2 = concatenate([up6_2, conv6_1], axis = 3)
        conv6_2 = standard_uint(conv6_2, nb_filter = 512)
        
        up5_3 = Conv2DTranspose(256, (2, 2), strides = (2, 2), padding = 'same')(conv6_2)
        conv5_3 = concatenate([up5_3, conv5_1, conv5_2], axis = 3)
        conv5_3 = standard_uint(conv5_3, nb_filter = 256)  
        
        up4_4 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_3)
        conv4_4 = concatenate([up4_4, conv4_1, conv4_2, conv4_3], axis = 3)
        conv4_4 = standard_uint(conv4_4, nb_filter = 128)     
        
        up3_5 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_4)
        conv3_5 = concatenate([up3_5, conv3_1, conv3_2, conv3_3, conv3_4], axis = 3)
        conv3_5 = standard_uint(conv3_5, nb_filter = 64)
        
        up2_6 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_5)
        conv2_6 = concatenate([up2_6, conv2_1, conv2_2, conv2_3, conv2_4, conv2_5], axis = 3)
        conv2_6 = standard_uint(conv2_6, nb_filter = 32)    
        
        up1_7 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_6)
        conv1_7 = concatenate([up1_7, conv1_1, conv1_2, conv1_3, conv1_4, conv1_5, conv1_6], axis = 3)
        conv1_7 = standard_uint(conv1_7, nb_filter = 16)
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        nested_output_5 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_6)
        nested_output_6 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_7)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4, nested_output_5, nested_output_6]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_6])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model


def get_nest_unet_512_layer_5(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)    
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_4])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model
    
    
    
def get_nest_unet_512_layer_5_more_channels(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)    
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_4])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model

In [ ]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


def weighted_dice_coeff(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight * weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    return score


def weighted_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = 1 - weighted_dice_coeff(y_true, y_pred, weight)
    return loss


def weighted_bce_loss(y_true, y_pred, weight):
    # avoiding overflow
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))

    # https://www.tensorflow.org/api_docs/python/tf/nn/weighted_cross_entropy_with_logits
    loss = (1. - y_true) * logit_y_pred + (1. + (weight - 1.) * y_true) * \
                                          (K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)


def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd number
    if K.int_shape(y_pred)[1] == 128:
        kernel_size = 11
    elif K.int_shape(y_pred)[1] == 256:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 512:
        kernel_size = 21
    elif K.int_shape(y_pred)[1] == 1024:
        kernel_size = 41
    else:
        raise ValueError('Unexpected image size')
    averaged_mask = K.pool2d(
        y_true, pool_size=(kernel_size, kernel_size), strides=(1, 1), padding='same', pool_mode='avg')
    border = K.cast(K.greater(averaged_mask, 0.005), 'float32') * K.cast(K.less(averaged_mask, 0.995), 'float32')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight += border * 2
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + (1 - weighted_dice_coeff(y_true, y_pred, weight))
    return loss

In [ ]:
def make_margin(imgs):
    
    """
        412の倍数のH,Wにする
    """
    
    original_image_shape = imgs.shape
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412) 
    made_imgs = np.zeros(
        (original_image_shape[0], H*412, W*412, 1), 
        np.float32
    )
    
    start_h = (H*412 - original_image_shape[1]) // 2
    start_w = (W*412 - original_image_shape[2]) // 2
    
    made_imgs[:, start_h:start_h+original_image_shape[1], start_w:start_w+original_image_shape[2], :] = imgs
    return made_imgs


def delete_margin(imgs, original_image_shape):
    
    """
        marginを削除
    """
    
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412)
    
    start_h = (H*412 - original_image_shape[1]) // 2
    start_w = (W*412 - original_image_shape[2]) // 2
    
    return imgs[:, start_h:start_h+original_image_shape[1], start_w:start_w+original_image_shape[2], :]

In [ ]:
# 学習後のU-Netによる予測を行う関数
def predict(X_test, model_path, filenames, out_dir, input_shape=(512, 512, 1), num_classes=1):

    # testDataフォルダ配下にleft_imagesフォルダを置いている
    
    model = get_nest_unet_512_layer_5(input_shape=input_shape, num_classes=num_classes)
    
    model.load_weights(model_path)
    BATCH_SIZE = 1
    Y_pred = model.predict(X_test, BATCH_SIZE)
    
    print(Y_pred.shape)
    os.makedirs(out_dir, exist_ok = True)
    
    if Y_pred.shape[3]!=1:
        num = Y_pred.shape[3]
        for n in range(num):
            os.makedirs(os.path.join(out_dir,str(n+1)), exist_ok=True)
        for i, y in enumerate(Y_pred):
            for n in range(num):
                os.makedirs(os.path.join(out_dir,str(n+1)), exist_ok=True)
            # testDataフォルダ配下にleft_imagesフォルダを置いている
            #img = cv2.imread('512/hira/For_Seika/learning/original/'+ file_names[i], cv2.IMREAD_GRAYSCALE)
            #y = cv2.resize(y, (img.shape[1], img.shape[0]))
                cv2.imwrite(os.path.join(out_dir, str(n+1) , str(i).zfill(6) + '.png'), denormalize_y(y[:,:,n]))
        
    else:
        for i, y in enumerate(Y_pred):
            # testDataフォルダ配下にleft_imagesフォルダを置いている
            #img = cv2.imread('512/hira/For_Seika/learning/original/'+ file_names[i], cv2.IMREAD_GRAYSCALE)
            #y = cv2.resize(y, (img.shape[1], img.shape[0]))
            cv2.imwrite(os.path.join(out_dir , str(i).zfill(6) + '.png'), denormalize_y(y))

# shCtrl

## cropped_001

### pred_001

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_002')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_002')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_002')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_002'
os.makedirs(path, exist_ok = True)

### pred_002

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_003')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_003')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_003')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/merged_mito_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_003'
os.makedirs(path, exist_ok = True)

### pred_003

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_001/mito_active_learning/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/merged_mito_004'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/modified_mito_004'
os.makedirs(path, exist_ok = True)

### pred_final

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_001/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(mito_imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    
    cnt += 1
    


## cropped_002

### pred_001

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_xy_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_yz_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_zx_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_xy_002')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_yz_002')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_zx_002')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/modified_mito_002'
os.makedirs(path, exist_ok = True)

### pred_002

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_002/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_xy_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_yz_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_zx_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_xy_003')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_yz_003')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_zx_003')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/merged_mito_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/modified_mito_003'
os.makedirs(path, exist_ok = True)

### pred_final

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_002/resize_10x10x10/')


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_xy_003')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_yz_003')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/pred_mito_zx_003')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_002/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(mito_imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    
    cnt += 1
    


## cropped_003

### pred_001

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_002')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_002')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_002')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/modified_mito_002'
os.makedirs(path, exist_ok = True)

### pred_002

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_003')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_003')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_003')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/merged_mito_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/modified_mito_003'
os.makedirs(path, exist_ok = True)

### pred_003

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_003/mito_active_learning/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/merged_mito_004'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/modified_mito_004'
os.makedirs(path, exist_ok = True)

### pred_final

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_003/resize_10x10x10/')


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs


imgs_xy, _ = load_Y_gray('Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_003/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(mito_imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    
    cnt += 1
    


## cropped_004

### pred_final

In [ ]:
model_path = 'Z:DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_004/resize_10x10x10/')
ori_image_shape = ori_imgs.shape


In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/pred_mito_xy_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/pred_mito_yz_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/pred_mito_zx_001/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/pred_mito_xy_001')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/pred_mito_yz_001')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/pred_mito_zx_001')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_004/modified_mito'
os.makedirs(out_dir, exist_ok = True)


cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


## cropped_005

### pred_001

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_005/resize_10x10x10/')


In [ ]:
## 初回だけ

# save_ori_imgs
ori_image_shape = ori_imgs.shape
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/original'
os.makedirs(out_dir, exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    img = ori_imgs[i][y_range[0]:y_range[1], x_range[0]:x_range[1], :]
    cv2.imwrite(
        f"{out_dir}/{str(cnt).zfill(4)}.png",
        denormalize_x(img)
    )
    cnt += 1
    
img.shape

In [ ]:
model_path = 'Z:DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_xy_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_yz_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_zx_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_xy_001')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_yz_001')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_zx_001')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx




In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_mito_001'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_005/modified_mito_001'
os.makedirs(path, exist_ok = True)

### pred_002

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shCtrl_003/dataset/cropped_005/resize_10x10x10/')


In [ ]:
model_path = 'Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_005/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
model_path = 'Z:DeepLearningData/research_010_NIH3T3/shCtrl_003/models/cropped_005/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_xy_002')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_yz_002')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz


imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/pred_mito_zx_002')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotatioins/cropped_005/modified_mito_002'
os.makedirs(path, exist_ok = True)

### pred_final

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shCtrl_003/annotations/cropped_005/modified_mito'
os.makedirs(out_dir, exist_ok = True)


cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1

    


# shOPA1

## cropped_001

### pred_001

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_002')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_002')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_002')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_002'
os.makedirs(path, exist_ok = True)

### pred_002

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_003')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_003')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_003')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/merged_mito_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_003'
os.makedirs(path, exist_ok = True)

### pred_003

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/merged_mito_004'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/modified_mito_004'
os.makedirs(path, exist_ok = True)

### pred_final

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_001/resize_10x10x10/')


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_001/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(mito_imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1
    


## cropped_002

### pred_001

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10/')
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_002')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_002')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_002')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_002'
os.makedirs(path, exist_ok = True)

### pred_002

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10/')
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_003')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_003')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_003')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/merged_mito_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_003'
os.makedirs(path, exist_ok = True)

### pred_003

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10/')
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_002/mito_active_learning/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_004/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test

In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/merged_mito_004'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(6)}.png', img_)
    
    cnt += 1
    


In [ ]:
path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/modified_mito_004'
os.makedirs(path, exist_ok = True)

### pred_final

In [ ]:
ori_imgs, filename =  load_X_gray('Z://DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_002/resize_10x10x10/')


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape


imgs_xy, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_xy_004')
merged_imgs_xy = merge_imgs(imgs_xy, ori_image_shape)
del imgs_xy


imgs_yz, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_yz_004')
merged_imgs_yz = merge_imgs(imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del imgs_yz



imgs_zx, _ = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/pred_mito_zx_004')
merged_imgs_zx = merge_imgs(imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del imgs_zx

In [ ]:
mito_imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotatioins/cropped_002/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(mito_imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    cnt += 1
    


## cropped_003

### pred_001

In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_003/mito_active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_003/resize_10x10x10/')
ori_image_shape = ori_imgs.shape


In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_xy_002')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_yz_002')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_zx_002')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_mito_002/'
os.makedirs(out_dir, exist_ok = True)

### pred_002

In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_003/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
ori_imgs, filename =  load_X_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/dataset/cropped_003/resize_10x10x10/')
ori_image_shape = ori_imgs.shape


In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_xy_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_yz_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_zx_003/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_xy_003')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_yz_003')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/pred_mito_zx_003')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_mito_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/manually_mito_003/'
os.makedirs(out_dir, exist_ok = True)

### pred_final

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_003/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    

    cnt += 1

    


## cropped_004

### pred_final

In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/pred_mito_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/pred_mito_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/pred_mito_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/pred_mito_xy_002')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/pred_mito_yz_002')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/pred_mito_zx_002')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_004/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    

    cnt += 1

    


## cropped_005

### pred_final

In [ ]:
model_path = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/models/cropped_001/mito_active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

target_imgs = []
target_imgs_lst = []
for i in range(seped_xy_imgs.shape[0]):
    if np.size(seped_xy_imgs[i]) != np.sum(seped_xy_imgs[i] == -1):
        target_imgs.append(seped_xy_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/pred_mito_xy_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_xy_imgs, X_test



seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_yz_imgs.shape[0]):
    if np.size(seped_yz_imgs[i]) != np.sum(seped_yz_imgs[i] == -1):
        target_imgs.append(seped_yz_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/pred_mito_yz_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_yz_imgs, X_test, target_imgs, target_imgs_lst


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

target_imgs = []
target_imgs_lst = []
for i in range(seped_zx_imgs.shape[0]):
    if np.size(seped_zx_imgs[i]) != np.sum(seped_zx_imgs[i] == -1):
        target_imgs.append(seped_zx_imgs[i])
        target_imgs_lst.append(str(i).zfill(6))
target_imgs = np.array(target_imgs)

X_test = target_imgs
out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/pred_mito_zx_002/'
predict_with_name(X_test, model_path, target_imgs_lst, out_dir, input_shape, num_classes)

del seped_zx_imgs, X_test, target_imgs, target_imgs_lst

In [ ]:
imgs_xy, imgs_xy_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/pred_mito_xy_002')

full_imgs_xy_sheets = ori_image_shape[0] * (-(-ori_image_shape[1] // 412)) * (-(-ori_image_shape[2] // 412))
full_imgs_xy = np.zeros((full_imgs_xy_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_xy, imgs_xy_filename):
    full_imgs_xy[int(imgname.split(".")[0])] = img

merged_imgs_xy = merge_imgs(full_imgs_xy, ori_image_shape)

del imgs_xy, full_imgs_xy


imgs_yz, imgs_yz_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/pred_mito_yz_002')

full_imgs_yz_sheets = ori_image_shape[2] * (-(-ori_image_shape[0] // 412)) * (-(-ori_image_shape[1] // 412))
full_imgs_yz = np.zeros((full_imgs_yz_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_yz, imgs_yz_filename):
    full_imgs_yz[int(imgname.split(".")[0])] = img

merged_imgs_yz = merge_imgs(full_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))

del imgs_yz, full_imgs_yz


imgs_zx, imgs_zx_filename = load_Y_gray('Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/pred_mito_zx_002')

full_imgs_zx_sheets = ori_image_shape[1] * (-(-ori_image_shape[2] // 412)) * (-(-ori_image_shape[0] // 412))
full_imgs_zx = np.zeros((full_imgs_zx_sheets, 512, 512, 1), np.float32)
for img, imgname in zip(imgs_zx, imgs_zx_filename):
    full_imgs_zx[int(imgname.split(".")[0])] = img

merged_imgs_zx = merge_imgs(full_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))

del imgs_zx, full_imgs_zx

In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_mito_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

    


In [ ]:
imgs_ave = merged_imgs_xy * 255 // 3 + merged_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_imgs_zx.transpose(2, 0, 1, 3) * 255 //3


out_dir = 'Z:/DeepLearningData/research_010_NIH3T3/shOPA1_003/annotations/cropped_005/merged_mito_final'
os.makedirs(out_dir, exist_ok = True)

cnt = 0
for i in range(imgs_ave.shape[0]):
    
    # threshed
    img = np.where(
        imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    

    cnt += 1

    
